# **Scopo del Progetto**

Lo scopo del progetto è quello di creare un servizio meteorologico in tempo reale, che permetta di visualizzare i dati meteo provenienti da più fonti pubbliche (con classificazione e previsione dei dati).
Al progetto abbiamo integrato (lato Spark) una funzionalità legata alla classificazione delle immagini che, in caso di immagine con possibile pioggia invia un messagio ad un canale telegram (gestito da un bot telegram), mentre nel caso in cui l'immagine sia considerata non valida invia una richiesta di reset al driver selenium lato producer. 
Nel progetto verranno utilizzati i seguenti applicativi:


*   Selenium Python
*   Fluentd
*   Apache Kafka
*   Apache Spark
*   Elasticsearch
*   Kibana

# **Breve descrizione**

- ***Selenium Python***
Per l'estrazione dei dati dalle fonti web pubbliche abbiamo deciso di utilizzare degli script in Python; in particolare abbiamo deciso di utilizzare la libreria Selenium Python: questa simula l'interazione umana con un browser web (click su un link, screenshot, etc..).

- ***Fluentd***
Fluentd è un software Open Source di data injestion, che consente di unificare la raccolta ed il consumo dei dati per una loro migliore comprensione ed utilizzo. Garantisce che i dati ottenuti dagli script python non vengano persi in caso di inoperatività di parti del sistema. Al suo interno vengono utilizzati i plugin nativi per l'inserimento dei dati provenienti dagli script Python, ed inoltre viene integrato un plugin per l'instradamento degli stessi verso Apache Kafka.

- ***Apache Kafka***
Kafka è una piattaforma Open Source di stream processing a bassa latenza ed alta velocità per la gestione di feed dati in tempo reale. E' in grado di ottenere dati da molteplici fonti e di instradarle su differenti destinazioni. Nel progetto, Kafka ottiene i dati da Fluentd, non effettua alcun processing su di essi, e li invia ad Apache Spark. Garantisce la resilienza dei dati e che questi arrivino a Spark nello stesso ordine in cui vengono ottenuti.

- ***Apache Spark***
Spark è un framework Open Source per il calcolo distribuito. Permette l'esecuzione di data engineering, machine learning, data science in locale o su cluster. Consente l'uso di vari linguaggi di programmazione e l'integrazione di software di terze parti (Elasticsearch), attraverso appositi plugin.
Nel progetto i dati provenienti da Kafka vengono letti attraverso il meccanismo di structured streaming, in tempo reale, con l'integrazione di un plugin Kafka.
I dati ottenuti verranno inseriti in un dataframe, verrà effettuata un'operazione di data regression su di essi in modo tale da ottenere una previsione statistica; in seguito verranno indicizzati su Elasticsearch e salvati in modo permanente sul filesystem locale.
Le immagini ottenute verranno inserite in un dataframe differente, verranno sottoposte ad un processo di image classification per stabilire se siano considerate "valide" e la classificazione ottenuta verrà indicizzata su Elasticsearch.
Nel caso l'immagine venga valutata come nuvoloso (il modello in uso permette correttamente di valutare solo 2 stati: valido e nd), dopo la seconda valutazione nuvoloso invia una messaggio al canale telegram gestito dal bot.
Nel caso l'immagine venga valutata nd, cioè non valida, viene inviata una richiesta di reset al producer python, che da Spark viene inviata a Kafka (topic restart_request) con valore "true", indicante di ricaricare il driver selenium relativo allo scraping dell'immagine meteo. Nel caso l'immagine sia considerata valida Spark invia comunque dati attraverso Kafka al producer python indicando la richiesta di reset come "false".

- ***Elasticsearch***
Elasticsearch è un server di ricerca, con capacità Full Text, con supporto ad architetture distribuite. Le informazioni vengono gestite come documenti JSON.
Tutti i dati trattati, che verranno successivamente visualizzati su Kibana, vengono indicizzati su Elasticsearch in modo tale da poter effettuare delle interrogazioni (query) in maniera efficiente.

- ***Kibana***
Kibana è un software di dashboard per la visualizzazione dei dati contenuti in Elasticsearch. Permette agli utenti di creare presentazioni dei dati sottoforma di grafici, tabelle, Canvas, e così via. Nel progetto si fa uso di alcuni grafici che mostrano i dati storici relativi alle rilevazioni, delle tabelle che mostrano le previsioni dei dati e le classificazioni delle immagini e dei grafici che mostrano i dati giornalieri.

<br>
<br> 

<p align="center">
    <img src="gitData/meme.png" alt="meme"/>
</p>